In [ ]:
import torch
import torch.nn.functional as F
from torch import autograd, nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import torchvision

In [ ]:
class ResidualBlock(nn.Module):

    def __init__(self, in_channel, out_channel, use_conv3=False, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=3, stride=stride, padding=1, bias=False)
        self.conv2 = nn.Conv2d(out_channel, out_channel, kernel_size=3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.bn2 = nn.BatchNorm2d(out_channel)
        if use_conv3:
          self.shortcut = nn.Conv2d(in_channel, out_channel, kernel_size=1, stride=stride, bias=False)
        else:
          self.shortcut = None

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if self.shortcut:
          x = self.shortcut(x)
        out = F.relu(out+x)
        return out

def resnet_block(in_channel, out_channel, num_residuals, first_block=False,stride=1): 
  blk = nn.Sequential() 
  for i in range(num_residuals): 
    block_name = 'block{}'.format(i+1)
    if i == 0:
      if not first_block: 
        blk.add_module(block_name,ResidualBlock(in_channel, out_channel, use_conv3=True, stride=stride))
      else:
        blk.add_module(block_name,ResidualBlock(in_channel, out_channel, use_conv3=False, stride=stride))
    else: 
      blk.add_module(block_name,ResidualBlock(out_channel, out_channel, stride=1)) 
  return blk

In [ ]:
def init_weights(module):
  if isinstance(module, nn.Linear):
    nn.init.xavier_normal_(module.weight.data)
  if isinstance(module, nn.Conv2d):
    nn.init.kaiming_normal_(module.weight.data, mode='fan_out')

class Resnet(nn.Module):
  def __init__(self, batch_size):
    super(Resnet, self).__init__()
    self.layers = nn.Sequential(
        nn.Conv2d(3, 64, 3, stride=1, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU()
        # nn.MaxPool2d(3, stride=2, padding=1)
    )
    self.stage1 = resnet_block(64,64,2,first_block=True,stride=1)
    self.stage2 = resnet_block(64,128,2,stride=2)
    self.stage3 = resnet_block(128,256,2,stride=2)
    # self.stage4 = resnet_block(64,128,2,stride=2)

    with torch.no_grad():
      self.conv_output_size = self._forward_sub(torch.zeros(batch_size,3,64,64)).view(batch_size, -1).shape[1]
    print("conv output size is ",self.conv_output_size)
    self.fc = nn.Linear(self.conv_output_size, 4000, bias=False)

    self.apply(init_weights)
  
  def _forward_sub(self, x):
    x = self.layers(x)
    x = self.stage1(x)
    x = self.stage2(x)
    x = self.stage3(x)
    x = F.adaptive_avg_pool2d(x, output_size=1)
    return x

  def forward(self, x):
    x = self._forward_sub(x)
    x = x.view(x.size(0), -1)
    x = self.fc(x)
    return x


In [ ]:
def xavier_init(params):
  for m in params:
    if isinstance(m,nn.Linear) or isinstance(m,nn.Conv2d):
      nn.init.xavier_normal_(m.weight)
      
class MyModel(nn.Module):
  def __init__(self):
    super(MyModel, self).__init__()

    self.conv_1 = nn.Conv2d(3,64,3,1,1)
    self.bn_1 = nn.BatchNorm2d(64)
    # self.do_1 = nn.Dropout2d(0.3)
    self.conv_2 = nn.Conv2d(64,64,3,1,1)
    self.bn_2 = nn.BatchNorm2d(64)
    self.pool_1 = nn.MaxPool2d(2,2)
    self.conv_3 = nn.Conv2d(64,64,3,1,1)
    self.bn_3 = nn.BatchNorm2d(64)
    self.pool_2 = nn.MaxPool2d(2,2)
    self.conv_4 = nn.Conv2d(64,128,3,1,1)
    self.bn_4 = nn.BatchNorm2d(128)
    self.do_2 = nn.Dropout2d(0.3)
    self.conv_5 = nn.Conv2d(128,128,3,1,1)
    self.bn_5 = nn.BatchNorm2d(128)
    self.pool_3 = nn.MaxPool2d(2,2)
    self.conv_6 = nn.Conv2d(128,256,3,1,1)
    self.bn_6 = nn.BatchNorm2d(256)
    self.do_3 = nn.Dropout2d(0.3)
    self.conv_7 = nn.Conv2d(256,256,3,1,1)
    self.bn_7 = nn.BatchNorm2d(256)
    self.pool_4 = nn.MaxPool2d(2,2)
    self.conv_8 = nn.Conv2d(256,256,3,1,1)
    self.bn_8 = nn.BatchNorm2d(256)
    self.pool_5 = nn.MaxPool2d(2,2)
    self.conv_9 = nn.Conv2d(256,256,3,1,1)
    self.bn_9 = nn.BatchNorm2d(256)
    self.pool_6 = nn.MaxPool2d(2,2)
    self.fc = nn.Linear(256,4000,bias=False)

  def forward(self, x):
    # x = self.do_1(F.relu(self.bn_1(self.conv_1(x))))
    x = F.relu(self.bn_1(self.conv_1(x)))
    x = F.relu(self.bn_2(self.conv_2(x)))
    x = self.pool_1(x)
    x = F.relu(self.bn_3(self.conv_3(x)))
    x = self.pool_2(x)
    x = self.do_2(F.relu(self.bn_4(self.conv_4(x))))
    x = F.relu(self.bn_5(self.conv_5(x)))
    x = self.pool_3(x)
    x = self.do_3(F.relu(self.bn_6(self.conv_6(x))))
    x = F.relu(self.bn_7(self.conv_7(x)))
    x = self.pool_4(x)
    x = F.relu(self.bn_8(self.conv_8(x)))
    x = self.pool_5(x)
    x = F.relu(self.bn_9(self.conv_9(x)))
    x = self.pool_6(x)
    # print(x.shape)
    x = x.view(-1,256)
    # print(x.shape)
    x = self.fc(x)    
    return x
  
  def init_weights(self):
    with torch.no_grad():
      xavier_init(self.modules())

In [ ]:
import time
def train(model, data_loader, test_loader, task='Classification'):
    model.train()

    for epoch in range(epochs):
      start_time = time.time()
      avg_loss = 0.0
      for batch_num, (features, labels) in enumerate(data_loader):
          features, labels = features.to(device), labels.to(device)
          
          optimizer.zero_grad()
          outputs = model(features)
          loss = criterion(outputs, labels.long())
          loss.backward()
          optimizer.step()
          
          avg_loss += loss.item()

          if batch_num % 400 == 399:
              print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch+1, batch_num+1, avg_loss/400))
              end_time = time.time()
              print(f"400 batches took {end_time - start_time} seconds")
              avg_loss = 0.0    
              start_time = time.time()
          
          torch.cuda.empty_cache()
          del features
          del labels
          del loss
      scheduler.step()
      if task == 'Classification':
          val_loss, val_acc = test_classify(model, test_loader)
          train_loss, train_acc = test_classify(model, data_loader)
          print('Train Loss: {:.4f}\tTrain Accuracy: {:.4f}\tVal Loss: {:.4f}\tVal Accuracy: {:.4f}'.
                format(train_loss, train_acc, val_loss, val_acc))
      else:
          test_verify(model, test_loader)
      
      
      torch.save({'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict' : scheduler.state_dict(),
      }, "/content/"+"Model_"+str(epoch))

def test_classify(model, test_loader):
  model.eval()
  test_loss = []
  accuracy = 0
  total = 0

  for batch_num, (features, labels) in enumerate(test_loader):
      features, labels = features.to(device), labels.to(device)
      outputs = model(features)
            
      _, pred_labels = torch.max(outputs,1)
      # pred_labels = pred_labels.view(-1)
      
      loss = criterion(outputs, labels.long())
      
      accuracy += torch.sum(torch.eq(pred_labels, labels)).item()
      total += len(labels)
      test_loss.extend([loss.item()]*features.size()[0])
      del features
      del labels

  model.train()
  return np.mean(test_loss), accuracy/total

def test_verify(model, test_loader):
  pass

In [ ]:
train_dataset = torchvision.datasets.ImageFolder(root='/content/classification_data/train_data',transform=torchvision.transforms.ToTensor())
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=8)
val_dataset = torchvision.datasets.ImageFolder(root='/content/classification_data/val_data',transform=torchvision.transforms.ToTensor())
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=8)
test_dataset = torchvision.datasets.ImageFolder(root='/content/classification_data/test_data',transform=torchvision.transforms.ToTensor())
test_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=8)

In [ ]:
print(train_dataset.__len__(), len(train_dataset.classes))
for x, y in train_dataloader:
  print(x.shape)
  break

380638 4000
torch.Size([256, 3, 64, 64])


In [ ]:
# Resnet
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
net = MyModel()
print(net)
net.to(device)
epochs = 20
learningRate = 0.15
weightDecay = 5e-5
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.85)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max',factor=0.1,verbose=True, patience=2)

cuda
MyModel(
  (conv_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn_2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn_3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool_2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn_4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (do_2): Dropout2d(p=0.3, inplace=False)
  (conv_5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn_5): BatchNo

In [ ]:
pytorch_total_params = sum([p.numel() for n, p in net.named_parameters() if "weight" in n])
print(pytorch_total_params)

3386496


In [ ]:
epochs = 5
temp = torch.load("Model_14")
net.load_state_dict(temp['model_state_dict'])
optimizer.load_state_dict(temp['optimizer_state_dict'])
scheduler.load_state_dict(temp['scheduler_state_dict'])

In [ ]:
# Baseline
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(device)
# net = MyModel()
# net.init_weights()
# print(net)
# net.to(device)
# epochs = 10
# learningRate = 0.15
# weightDecay = 5e-5
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(net.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.85)

In [ ]:
train(net, train_dataloader, test_dataloader) 

In [ ]:
from torch.utils.data import Dataset
from PIL import Image

class VerifyDataset(Dataset):
  def __init__(self, dir):
    data_file = open(dir)
    self.file_list = []
    lines = data_file.readlines()
    for line in lines:
      img1, img2 = line.strip('\n').split(' ')
      self.file_list.append((img1, img2))

  def __len__(self):
    return len(self.file_list)

  def __getitem__(self, index):
    img1_dir, img2_dir = self.file_list[index]
    img1 = Image.open(img1_dir)
    img2 = Image.open(img2_dir)
    img1 = torchvision.transforms.ToTensor()(img1)
    img2 = torchvision.transforms.ToTensor()(img2)
    return img1, img2

veri_dataset = VerifyDataset("/content/verification_pairs_test.txt")
veri_dataloader = torch.utils.data.DataLoader(veri_dataset, batch_size=10, shuffle=False, num_workers=1)
cos = nn.CosineSimilarity(dim=1, eps=1e-6)

def verify(model, dataloader):
  model.eval()

  for i, (img1, img2) in enumerate(dataloader):
    img1 = img1.to(device)
    img2 = img2.to(device)

    emb_1 = model(img1)
    emb_2 = model(img2)

    similarity = cos(emb_1, emb_2)
    similarity.view(-1, 1)
    res.append(similarity.cpu().tolist())

In [ ]:
import copy
verify_net = copy.deepcopy(net)
verify_net.fc = nn.Sequential()
res = []
verify(verify_net, veri_dataloader)

In [ ]:
ans = sum(res, [])
print(len(ans))

51835


In [ ]:
id = [file1+' '+file2 for (file1, file2) in veri_dataset.file_list]
print(id[0])

verification_data/00020839.jpg verification_data/00035322.jpg


In [ ]:
import pandas as pd
d = {"Id":id, "Category":ans}
df = pd.DataFrame(d)
df.to_csv("submission.csv", index=False)